In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset = pd.get_dummies(dataset,dtype=int, drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
dataset = dataset.drop("User ID", axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [9]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [11]:
independent = dataset[["Age","EstimatedSalary","Gender_Male"]]
dependent = dataset["Purchased"]

In [13]:
independent.shape

(400, 3)

In [15]:
dependent

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [17]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [25]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

#Here the fit.transform function Standardize the Undefined data to Defined Data and make it to be saved in X_train
X_train = sc.fit_transform(X_train)

#Here the transform function uses the process occurred in fit.transform function for the input X_test and make it to be saved in X_train
X_test = sc.transform(X_test)

In [27]:
from sklearn.model_selection import GridSearchCV #Here cv refers to Cross Validation
from sklearn.svm import SVC

#Creating a Dictionary for Playable Parameters
param_grid = {'kernel':['linear','rbf','poly','sigmoid'], 'gamma':['auto','scale'], 'C':[10,100,1000,2000,3000]}

#if refit = false then final model will be considered as Last Combination of parameter_grid like (linear,3000,scale)
#n_jobs = -1 refers to Run all the Models Parallely
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3, cv=5, n_jobs=-1, scoring='f1_weighted') 

#All the above steps are called "Procedure Load"

#Model Creation
grid.fit(X_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [47]:
# print best parameter after tuning 
#print(grid.best_params_) 
results = grid.cv_results_
#print(re)

In [31]:
y_pred = grid.predict(X_test)

In [33]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[80  5]
 [ 7 42]]


In [35]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.92      0.94      0.93        85
           1       0.89      0.86      0.88        49

    accuracy                           0.91       134
   macro avg       0.91      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134



In [39]:
from sklearn.metrics import f1_score
f1_macro = f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}: 0.9100355779243318


In [41]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9539015606242497

In [49]:
#To make All the Results into Tabular Format

table = pd.DataFrame.from_dict(results)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.017859,0.002335,0.006383,0.002297,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,21
1,0.013323,0.000761,0.005564,0.000965,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,3
2,0.014396,0.001702,0.004635,0.000483,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.799620,0.808392,0.787943,0.924528,0.901744,0.844445,0.056918,11
3,0.009308,0.000935,0.004464,0.000554,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.762677,0.738916,0.655795,0.796284,0.766556,0.744045,0.047743,39
4,0.015824,0.000990,0.004207,0.001133,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,21
5,0.014933,0.003169,0.008399,0.003803,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,3
6,0.021767,0.004036,0.003926,0.002351,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.799620,0.808392,0.787943,0.924528,0.901744,0.844445,0.056918,11
7,0.013914,0.001418,0.004350,0.000635,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.762677,0.738916,0.753180,0.778067,0.766556,0.759879,0.013172,31
8,0.052898,0.007182,0.003145,0.001144,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,21
9,0.017032,0.002077,0.004762,0.001989,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.867478,0.886792,0.870362,0.944161,0.943041,0.902367,0.034308,1


In [53]:
age_input=float(input("Age:"))
salary_input=float(input("BMI:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 35
BMI: 40
Sex Male 0 or 1: 1


In [55]:
#prediction
#Keyword - predict is function of Application 
Future_Prediction = grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction{}:".format(Future_Prediction))

Future_Prediction[1]:


In [37]:
#dir(clf_report)

In [57]:
# Naming the Model with its extension .sav
import pickle
filename = "finalized_model_SVM_Grid_classification_Assignment.sav"

#Model will be saved in respective path, once after the below Code Execution
# Keyword - wb refers to "write binary"
pickle.dump(grid,open(filename,'wb'))

In [59]:
# Load the Saved Model and check the same to send it for Phase-02 (Deployment)
# # Keyword - rb refers to "read binary"

loaded_model=pickle.load(open("finalized_model_SVM_Grid_classification_Assignment.sav",'rb'))
result = loaded_model.predict([[age_input,salary_input,sex_male_input]]) 
result

array([1], dtype=int64)